In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil
import sys
import matplotlib 
matplotlib.use('Agg')

from tqdm import tqdm

In [2]:
from tactile_learning.datasets import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
def get_kinova_states(hdf5_path):
    with h5py.File(hdf5_path, 'r') as depth_file:
        # print(depth_file.keys())
        torque=depth_file['efforts'][()]
        # Printing all values of array without truncation
        np.set_printoptions(threshold=sys.maxsize)
        return torque

In [4]:
def plot_torque_change(data_path, joint, list_demo): 
    #Get torques from the demo
    print("-----------------------------------------------------")
    torque_lst = []
    for demo in list_demo:
        hdf5_path = f'{data_path}/demonstration_{demo}/allegro_joint_states.h5'
        

    #plot the torque, each joiint a plot
    joint_num = joint.count(1)
    
    figure, axis = plt.subplots(joint_num//3+1, 3)
    figure.set_size_inches(18.5, 10.5, forward=True)
    
    row = 0
    column = 0
    for j in range(len(joint)):
        if not joint[j]: continue
        print(j)
        for demo in list_demo: 
            hdf5_path = f'{data_path}/demonstration_{demo}/allegro_joint_states.h5'
            torque = get_kinova_states(hdf5_path)
            # print(row, column)
            axis[row,column].plot(torque[:,j])
        
        axis[row,column].set_title("lalaland")
        axis[row,column].legend(['demo_1', 'demo_2', 'demo_3', 'demo_4'])
            
        column += 1
        if column == 3: 
            row += 1
            column = 0

        figure.savefig("/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/torque.png")
            
        

In [5]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps, view_num=0): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, f'cam_{view_num}_rgb_video.metadata')
    allegro_torque_path = os.path.join(root, 'allegro_joint_states.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(allegro_torque_path, 'r') as f:
        allegro_timestamps = f['timestamps'][()]

    image_id, allegro_id = 0, 0
    curr_timestamp = allegro_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    allegro_indices, image_indices = [], []
    allegro_indices.append(allegro_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        allegro_id = get_closest_id(allegro_id, curr_timestamp, allegro_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > allegro_timestamps[allegro_id] and curr_timestamp > image_timestamps[image_id]:
            break

        allegro_indices.append(allegro_id)
        image_indices.append(image_id)

    assert len(allegro_indices) == len(image_indices)
    return allegro_indices, image_indices


In [20]:

def dump_states(root, allegro_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(allegro_indices))

    with h5py.File(os.path.join(root, 'allegro_joint_states.h5'), 'r') as f:
        all_joint_torques = f['efforts'][()]


    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    # frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(allegro_indices)):
        allegro_id, image_id = allegro_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        # print('allegro_indices[:allegro_id+1].shape: {}'.format(
        #     len(allegro_indices[:i+1])
        # ))
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            torque_values = all_joint_torques[allegro_indices[:i+1]],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def plot_torque(ax, torque_values, title):
    
    ax.set_title(title)
    ax.set_ylim(-1.3, 1.3)
    # print('torque_values in plot_torque: {}'.format(torque_values.shape))
    ax.plot(torque_values)

def map_segment_to_joint(segment_id):
    # Joint order: [index_top, index_mid, index_mid2, index_tip,
    #               mid_top, mid_mid, mid_mid2, mid_tip,
    #               ring_top, ring_mid, ring_mid2, ring_tip, 
    #               thumb_top, thumb_mid, thumb_mid2, thumb_tip]
    # Segment order: []

    segment_to_joint = {
        0: 15, # Tips
        1: 3,
        2: 7,
        3: 11,
        4: 14, # Second segment 
        5: 2, 
        6: 6, 
        7: 10,
        8: 13, # Third segment
        9: 1,
        10: 5,
        11: 9,
        12: 12, # Knuckles,
        13: 0,
        14: 4,
        15: 8
    }

    # print('segment_id: {}'.format(
    #     segment_id
    # ))
    return segment_to_joint[segment_id]

def dump_demo_state(frame_id, viz_dir, torque_values, camera_img):
    # tactile_values: (15,16,3)
    # print('torque_values.shape in dump_demo_state: {}'.format(
    #     torque_values.shape
    # ))
    
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            # if row_id + column_id > 0: # The top left axis should stay empty
            torque_joint_id = map_segment_to_joint(row_id*4 + column_id)
            plot_torque(
                ax = axs[column_id][row_id],
                torque_values = torque_values[:,torque_joint_id],
                title = f'Torque in Joint {row_id*4+column_id}'
            )
    fig.suptitle('Torques Applied')
    fig.savefig(os.path.join(viz_dir, 'torque_state.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'torque_state.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [27]:
import glob
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling'
roots = glob.glob(f'{data_path}/demonstration_*')

roots = [
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_9',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_8',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_2',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_11',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_12'
]

retrieval_fps = 5
view_num = 0
for root in roots:
    allegro_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps, view_num=view_num)
    print('allegro_indices: {}, image_indices: {}'.format(allegro_indices, image_indices))
    dump_states(root, allegro_indices, image_indices, view_num=view_num)


allegro_indices: [0, 70, 120, 182, 246, 300, 361, 417, 480, 543, 604, 661, 724, 783, 841, 912, 963, 1023, 1078, 1138, 1201, 1260, 1318, 1382, 1440, 1500, 1557, 1616, 1674, 1738, 1801, 1856, 1913, 1976, 2034, 2095, 2154, 2215, 2272, 2342, 2393, 2453, 2518, 2577, 2637, 2692, 2783, 2811, 2875, 2933, 2992, 3055, 3114, 3171, 3234, 3295, 3351, 3410, 3471, 3531, 3606, 3650, 3713, 3774, 3830, 3889, 3947, 4010, 4069, 4133, 4189, 4247, 4310, 4368, 4426, 4484, 4547, 4606, 4668, 4729, 4785, 4846, 4907, 4967, 5025, 5083, 5146, 5203, 5270, 5323, 5383, 5442, 5507, 5566, 5625, 5683, 5741, 5801, 5870, 5924, 5983, 6042, 6100, 6161, 6220, 6281, 6340, 6404, 6473, 6520, 6580, 6639, 6698, 6759, 6818, 6878, 6940, 6998, 7058, 7118, 7176, 7241, 7297, 7354, 7415, 7476, 7535, 7594, 7655, 7718, 7779, 7835, 7894, 7955, 8017, 8079, 8133, 8193, 8253, 8311, 8373, 8432, 8494, 8555, 8613, 8673, 8736, 8791, 8850, 8909, 8973, 9034, 9087, 9150, 9206, 9273, 9345, 9390, 9449, 9508, 9568, 9631, 9688, 9755, 9808, 9871, 9927, 

100%|██████████| 359/359 [06:02<00:00,  1.01s/it]


In [24]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_11


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstration_12


[libx264 @ 0x55f27cc44bc0] using SAR=1/1
[libx264 @ 0x55f27cc44bc0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55f27cc44bc0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55f27cc44bc0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/torque_test/demonstratio

In [21]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
